In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/My Drive/Colab Notebooks/OCT classification/Ensemble/Distribution Ensemble/

/content/drive/My Drive/Colab Notebooks/OCT classification/Ensemble/Distribution Ensemble


In [3]:
!pip install -r requirements.txt --use-deprecated=legacy-resolver


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 14.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinu

In [ ]:
#%cd /content/drive/My Drive/KHOA HTTT/Giảng dạy HTTT/KLTN/Hướng dẫn/2024/KLTN - Phương, Quỳnh 2020 - OCT/

In [ ]:
!pip install numpy==1.26.3 --use-deprecated=legacy-resolver


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 53.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's legacy dependency resolver does not consider dependency conflicts when selecting packages. This behaviour is the source of the following dependency conflicts.
torchtext 0.18.0 requires torch>=2.3.0, but you'll have torch 2.2.1 which is incompatible.


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import os
import copy
import csv

In [5]:
# full test
dir ="/content/drive/My Drive/KHOA HTTT/Giảng dạy HTTT/KLTN/Hướng dẫn/2024/KLTN - Phương, Quỳnh 2020 - OCT"

# test 625
dir ="/content/drive/My Drive/Colab Notebooks/OCT classification/Ensemble"
data_dir = dir +"/Dataset OCT"
mean = np.array([0.19092108, 0.19092108, 0.19092108])
std = np.array([0.20110247, 0.20110247, 0.20110247])
batch_size = 32
num_workers = 32


data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((299, 299)),  # Inception v3 requires input size of 299x299
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'test': transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [6]:
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['test']}
testloader = torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size)

In [7]:
# Chuẩn bị dữ liệu

# Lấy dữ liệu từ dataloader và chuyển đổi thành tensor
x_observed = torch.tensor([])  # Khởi tạo tensor rỗng

for images, labels in testloader:
    x_observed = torch.cat((x_observed, images.flatten(start_dim=1)), dim=0)


print (x_observed)
len(x_observed)

tensor([[ 3.8282,  0.2206, -0.1889,  ...,  4.0232,  4.0232,  4.0232],
        [ 0.2401,  0.0451, -0.0134,  ...,  3.9452,  3.9452,  3.9452],
        [ 3.8672,  0.9227, -0.3449,  ...,  4.0232,  4.0232,  4.0232],
        ...,
        [ 4.0232,  4.0232,  4.0232,  ...,  4.0232,  4.0232,  4.0232],
        [-0.0914,  0.0841, -0.0134,  ..., -0.8909,  2.7362,  3.9647],
        [ 4.0232,  4.0232,  4.0232,  ..., -0.6374, -0.6959, -0.5399]])


625

# **Gompertz Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hàm Gompertz
def gompertz_function(x, eta, b):
    return 1 - torch.exp(-eta * (torch.exp(-b*x)-1))
    #return 1 - torch.exp(-eta * (torch.exp(-b*x)))
# Hàm hợp lý
def likelihood(params, x_observed):
    eta, b = params
    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(gompertz_function(x_feature, eta, b)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [1.0, 1.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    #print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()
    print (epoch, params, loss)

estimated_eta, estimated_b = params.tolist()

print("Hệ số eta ước lượng từ MLE:", estimated_eta)
print("Hệ số b ước lượng từ MLE:", estimated_b)


0 tensor([1.0100, 1.0100], requires_grad=True) tensor(nan, grad_fn=<SumBackward0>)
1 tensor([1.0200, 1.0200], requires_grad=True) tensor(nan, grad_fn=<SumBackward0>)
2 tensor([1.0300, 1.0300], requires_grad=True) tensor(nan, grad_fn=<SumBackward0>)
3 tensor([1.0400, 1.0400], requires_grad=True) tensor(nan, grad_fn=<SumBackward0>)
4 tensor([1.0499, 1.0499], requires_grad=True) tensor(nan, grad_fn=<SumBackward0>)
5 tensor([1.0599, 1.0599], requires_grad=True) tensor(nan, grad_fn=<SumBackward0>)
6 tensor([1.0698, 1.0698], requires_grad=True) tensor(nan, grad_fn=<SumBackward0>)
7 tensor([1.0798, 1.0797], requires_grad=True) tensor(nan, grad_fn=<SumBackward0>)
8 tensor([1.0897, 1.0896], requires_grad=True) tensor(nan, grad_fn=<SumBackward0>)
9 tensor([1.0996, 1.0995], requires_grad=True) tensor(nan, grad_fn=<SumBackward0>)
10 tensor([1.1094, 1.1093], requires_grad=True) tensor(nan, grad_fn=<SumBackward0>)
11 tensor([1.1193, 1.1191], requires_grad=True) tensor(nan, grad_fn=<SumBackward0>)
12

In [ ]:
import torch

# Hàm Gompertz
def gompertz(params, x):
    eta, b = params
    #return eta * b * torch.exp(-b * x_observed) * torch.exp(-eta * (1 - torch.exp(-b * x_observed)))
    return 1 - torch.exp(-eta * (torch.exp(-b*x)))

# Negative log-likelihood function
def negative_log_likelihood(params, x_observed):
    return -torch.sum(torch.log(gompertz(params, x_observed)))

# Dữ liệu quan sát được (ví dụ)
#data = torch.tensor([1, 2, 3, 4, 5], dtype=torch.float32)

# Ước lượng tham số bằng MLE
params = torch.tensor([1.0, 1.0], requires_grad=True)  # Giả định ban đầu cho eta và b

# Tối ưu hóa bằng phương pháp gradient descent
optimizer = torch.optim.SGD([params], lr=0.01)  # Sử dụng stochastic gradient descent (SGD) với learning rate là 0.01
num_iterations = 10  # Số lần lặp
for i in range(num_iterations):
    optimizer.zero_grad()
    loss = negative_log_likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

# Lấy kết quả ước lượng
eta_mle, b_mle = params.detach().numpy()
estimated_eta, estimated_b = params.tolist()

print("Tham số ước lượng bằng MLE:")
print("Hệ số eta ước lượng từ MLE:", estimated_eta)
print("Hệ số b ước lượng từ MLE:", estimated_b)

print("Eta:", eta_mle)
print("b:", b_mle)


Tham số ước lượng bằng MLE:
Eta: nan
b: nan


In [ ]:
import torch
from scipy.optimize import minimize

# Hàm Gompertz
def gompertz(params, x):
    eta, b = params
    #return eta * b * torch.exp(-b * x_observed) * torch.exp(-eta * (1 - torch.exp(-b * x_observed)))
    return 1 - torch.exp(-eta * (torch.exp(-b*x)))
# Negative log-likelihood function
def negative_log_likelihood(params, x_observed):
    return -torch.sum(torch.log(gompertz(params, x_observed)))


# Ước lượng tham số bằng MLE
initial_guess = torch.tensor([1.0, 1.0], requires_grad=True)  # Giả định ban đầu cho eta và b
result = minimize(negative_log_likelihood, initial_guess.detach().numpy(), args=(x_observed,), method='Nelder-Mead', bounds=[(0, None), (0, None)])

# Lấy kết quả ước lượng
eta_mle, b_mle = result.x

print("Tham số ước lượng bằng MLE:")
print("Eta:", eta_mle)
print("b:", b_mle)


AttributeError: module 'numpy.linalg._umath_linalg' has no attribute '_ilp64'

# **Shift Gompertz Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hàm Shift Gompertz
def shift_gompertz_function(x, eta, b):
    return (1- torch.exp(-b * x)) * torch.exp(-eta * torch.exp(-b * x)) # Shitf Gompertz

# Hàm hợp lý
def likelihood(params, x_observed):
    eta, b = params
    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(shift_gompertz_function(x_feature, eta, b)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [1.0, 1.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    #print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

estimated_eta, estimated_b = params.tolist()

print("Hệ số eta ước lượng từ MLE:", estimated_eta)
print("Hệ số b ước lượng từ MLE:", estimated_b)


Hệ số eta ước lượng từ MLE: -0.08309227228164673
Hệ số b ước lượng từ MLE: 2.0361244678497314


# **Weighted Gompertz Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hàm Weighted Gompertz
def weighted_gompertz_function(x, s, l):
    #return 1 - (1+(s * (torch.exp(l*x)-1))/(1+l*s))* torch.exp(-s * (torch.exp(l*x)-1))
    return 1 - (1+(s * (torch.exp(l*x)))/(1+l*s))* torch.exp(-s * (torch.exp(l*x)))
# Hàm hợp lý
def likelihood(params, x_observed):
    s, l = params
    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(weighted_gompertz_function(x_feature, s, l)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [1.0, 1.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    #print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

estimated_s, estimated_l = params.tolist()

print("Hệ số s ước lượng từ MLE:", estimated_s)
print("Hệ số l ước lượng từ MLE:", estimated_l)


Hệ số s ước lượng từ MLE: 1.8324379920959473
Hệ số l ước lượng từ MLE: 0.5636825561523438


# **Ibrahim Distribution Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hàm Ibrahim Distribution
def Ibrahim_function(x, l, b):
    return 1 - (1/(l+1))*(torch.exp(-b*x)+l*torch.exp(-(b*x)**2))


# Hàm hợp lý
def likelihood(params, x_observed):
    #p = params
    l, b = torch.abs(params)  # Đảm bảo p luôn dương
    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(Ibrahim_function(x_feature, l, b)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [2.0, 2.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    #print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

estimated_l, estimated_b = params.tolist()

print("Hệ số l ước lượng từ MLE:", estimated_l)
print("Hệ số b ước lượng từ MLE:", estimated_b)

Hệ số l ước lượng từ MLE: 1.0853286981582642
Hệ số b ước lượng từ MLE: 2.9221911430358887


# **New Burr Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hàm New Burr Gompertz
def new_burr_function(x, p):
    return (1+ torch.exp(-x**3))**(-p)

# Hàm hợp lý
def likelihood(params, x_observed):
    #p = params
    p = torch.abs(params)  # Đảm bảo p luôn dương
    if p <= 0:
        return torch.tensor(float('inf'))  # Trả về vô cùng dương nếu p không hợp lệ

    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(new_burr_function(x_feature, p)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [10.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    #print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

estimated_p = params.tolist()

print("Hệ số p ước lượng từ MLE:", estimated_p)


Hệ số p ước lượng từ MLE: [8.999977111816406]


# **Mitscherlich Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hàm  Mitscherlich
def Mitscherlich_function(x, a):
    #return a*(1- torch.exp(-c * (x+b)))
    return a*(1- a**(x-1))
# Hàm hợp lý
def likelihood(params, x_observed):
    a = params
    #a = torch.abs(params)  # Đảm bảo p luôn dương

    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(Mitscherlich_function(x_feature, a)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [1.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    #print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

#estimated_a, estimated_c, estimated_b = params.tolist()
estimated_a = params.tolist()
print("Hệ số a ước lượng từ MLE:", estimated_a)
#print("Hệ số c ước lượng từ MLE:", estimated_c)
#print("Hệ số b ước lượng từ MLE:", estimated_b)


Hệ số a ước lượng từ MLE: [nan]


# **logistic distribution Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hàm logistic distribution
def logistic_function(x, b0, b1):
    return 1/(1+ torch.exp(-(b0+ b1 *x)))

# Hàm hợp lý
def likelihood(params, x_observed):
    #p = params
    b0, b1 = torch.abs(params)  # Đảm bảo p luôn dương
    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(logistic_function(x_feature, b0, b1)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [10.0, 10.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    #print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

estimated_b0,estimated_b1  = params.tolist()

print("Hệ số b0 ước lượng từ MLE:", estimated_b0)
print("Hệ số b1 ước lượng từ MLE:", estimated_b1)

Hệ số b0 ước lượng từ MLE: 10.775835990905762
Hệ số b1 ước lượng từ MLE: 9.2244291305542


# **Exponential Distribution Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hàm Exponential Distribution
def exponential_function(x, l):
    return 1 - torch.exp(-l * x) # Exponential

# Hàm hợp lý
def likelihood(params, x_observed):
    l = params
    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(exponential_function(x_feature, l)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [1.0]  # Đoán giá trị ban đầu cho l
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    #print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

estimated_l = params.tolist()

print("Hệ số lambda ước lượng từ MLE:", estimated_l)


Hệ số lambda ước lượng từ MLE: [1.908218264579773]


# **Gamma Function**

In [14]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import scipy.special

# Hàm Gamma
def gamma_function(x, b):
    return scipy.special.gamma(x+b)

# Hàm hợp lý
def likelihood(params, x_observed):
    b = params
    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(gamma_function(x_feature, b)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [1.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    #print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

estimated_b = params.tolist()

print("Hệ số b ước lượng từ MLE:", estimated_b)


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

# **Power Gompertz Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hàm Power Gompertz
def power_gompertz_function(x, eta, b, t):
    return 1 - torch.exp(- eta * (torch.exp(b * (x ** t)) -1))

# Hàm hợp lý
def likelihood(params, x_observed):
    eta, b, t = params
    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(power_gompertz_function(x_feature, eta, b, t)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [1.0, 1.0, 1.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    #print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

estimated_eta, estimated_b, estimated_t = params.tolist()

print("Hệ số eta ước lượng từ MLE:", estimated_eta)
print("Hệ số b ước lượng từ MLE:", estimated_b)
print("Hệ số t ước lượng từ MLE:", estimated_t)


Hệ số eta ước lượng từ MLE: nan
Hệ số b ước lượng từ MLE: nan
Hệ số t ước lượng từ MLE: nan


# **Rayleigh Gamma Gompertz Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hàm Rayleigh Gamma Gompertz
def rayleigh_gamma_gompertz_function(x, t, l):
    return torch.exp(- (t/2) * (-torch.log(1- torch.exp(-l*x)))**2)

# Hàm hợp lý
def likelihood(params, x_observed):
    t, l = params
    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(rayleigh_gamma_gompertz_function(x_feature, t, l)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [1.0, 1.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    #print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

estimated_t, estimated_l = params.tolist()

print("Hệ số t ước lượng từ MLE:", estimated_t)
print("Hệ số l ước lượng từ MLE:", estimated_l)


Hệ số t ước lượng từ MLE: nan
Hệ số l ước lượng từ MLE: nan


# **Kumaraswamy Gompertz Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hàm Kumaraswamy Gompertz
def kumaraswamy_gompertz_function(x, t, g, a, b):
    return 1- (1- (1- torch.exp(-(t/g)* (torch.exp(g*x)-1)))** a)** b

# Hàm hợp lý
def likelihood(params, x_observed):
    t, g, a, b = params
    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(kumaraswamy_gompertz_function(x_feature, t, g, a, b)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [1.0, 1.0, 1.0, 1.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    #print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

estimated_t, estimated_g, estimated_a, estimated_b = params.tolist()

print("Hệ số t ước lượng từ MLE:", estimated_t)
print("Hệ số g ước lượng từ MLE:", estimated_g)
print("Hệ số a ước lượng từ MLE:", estimated_a)
print("Hệ số b ước lượng từ MLE:", estimated_b)


Hệ số t ước lượng từ MLE: nan
Hệ số g ước lượng từ MLE: nan
Hệ số a ước lượng từ MLE: nan
Hệ số b ước lượng từ MLE: nan


# **Bivariate Odd Generalized Exponential Gompertz Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hàm Bivariate Odd Generalized Exponential Gompertz
def BOGE_gompertz_function(x, a, b, c, l):
    return (1 - torch.exp(-a * (torch.exp((l/c)* (torch.exp(c*x)-1))-1)))**b

# Hàm hợp lý
def likelihood(params, x_observed):
    a, b, c, l = params
    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(BOGE_gompertz_function(x_feature, a, b, c, l)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [1.0, 1.0,1.0, 1.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    #print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

estimated_a, estimated_b, estimated_c, estimated_l = params.tolist()

print("Hệ số a ước lượng từ MLE:", estimated_a)
print("Hệ số b ước lượng từ MLE:", estimated_b)
print("Hệ số c ước lượng từ MLE:", estimated_c)
print("Hệ số l ước lượng từ MLE:", estimated_l)

Hệ số a ước lượng từ MLE: nan
Hệ số b ước lượng từ MLE: nan
Hệ số c ước lượng từ MLE: nan
Hệ số l ước lượng từ MLE: nan


# **Gompertz Inverse Exponential Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hàm Gompertz Inverse Exponential
def gompertz_inverse_exponential_function(x, a, b, t):
    return 1 - torch.exp((a/b) * (1-(1-(torch.exp(-t/x))**-b)))

# Hàm hợp lý
def likelihood(params, x_observed):
    a, b, t = params
    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(gompertz_inverse_exponential_function(x_feature, a, b, t)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [1.0, 1.0, 1.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    #print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

estimated_e, estimated_b, estimated_t = params.tolist()

print("Hệ số a ước lượng từ MLE:", estimated_a)
print("Hệ số b ước lượng từ MLE:", estimated_b)
print("Hệ số t ước lượng từ MLE:", estimated_t)



Hệ số a ước lượng từ MLE: nan
Hệ số b ước lượng từ MLE: nan
Hệ số t ước lượng từ MLE: nan


# **Unit-Gompertz Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Hàm Unit-Gompertz
def unit_gompertz_function(x, eta, b):
    return torch.exp(-eta * (x**(-b)-1))

# Hàm hợp lý
def likelihood(params, x_observed):
    eta, b = params
    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(unit_gompertz_function(x_feature, eta, b)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [1.0, 1.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

estimated_eta, estimated_b = params.tolist()

print("Hệ số eta ước lượng từ MLE:", estimated_eta)
print("Hệ số b ước lượng từ MLE:", estimated_b)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Hệ số eta ước lượng từ MLE: nan
Hệ số b ước lượng từ MLE: nan


# **Gompertz Lomax  Distribution Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


# Hàm Gompertz Lomax (GoLo)
def golo_function(x, a, b, g, t):
    return 1 - torch.exp((t/g)* (1 - pow(1+b*x,a*g)))

# Hàm hợp lý
def likelihood(params, x_observed):
    a, b, g, t = params
    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(golo_function(x_feature, a, b, g, t)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [1.0, 1.0, 1.0, 1.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

estimated_a, estimated_b, estimated_g, estimated_t = params.tolist()

print("Hệ số a ước lượng từ MLE:", estimated_a)
print("Hệ số b ước lượng từ MLE:", estimated_b)
print("Hệ số g ước lượng từ MLE:", estimated_g)
print("Hệ số t ước lượng từ MLE:", estimated_t)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Hệ số a ước lượng từ MLE: nan
Hệ số b ước lượng từ MLE: nan
Hệ số g ước lượng từ MLE: nan
Hệ số t ước lượng từ MLE: nan


# **Gompertz Fréchet (GoFr)  Distribution Function**

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


# Hàm Gompertz Fréchet (GoFr)
def gofr_function(x, a, b, g, t):
    return 1 - torch.exp((t/g)* (1 - pow(1-torch.exp(-pow(a/x, b)),-g)))

# Hàm hợp lý
def likelihood(params, x_observed):
    a, b, g, t = params
    # Tính giá trị hợp lý cho mỗi giá trị quan sát được
    likelihoods = [-torch.log(gofr_function(x_feature, a, b, g, t)) for x_feature in x_observed]
    # Tổng hợp tất cả các giá trị hợp lý
    total_likelihood = torch.sum(torch.stack(likelihoods))
    return total_likelihood

# Ước lượng tham số bằng phương pháp MLE
initial_guess = [1.0, 1.0, 1.0, 1.0]  # Đoán giá trị ban đầu cho eta và b
params = torch.tensor(initial_guess, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.01)

# Huấn luyện mô hình
for epoch in range(100):
    print (epoch)
    optimizer.zero_grad()
    loss = likelihood(params, x_observed)
    loss.backward()
    optimizer.step()

estimated_a, estimated_b, estimated_g, estimated_t = params.tolist()

print("Hệ số a ước lượng từ MLE:", estimated_a)
print("Hệ số b ước lượng từ MLE:", estimated_b)
print("Hệ số g ước lượng từ MLE:", estimated_g)
print("Hệ số t ước lượng từ MLE:", estimated_t)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
